# pipeline

pipeline可以根据传入的pipeline对象和起始还有截止日期返回一个多层索引的对象,pipeline是根据传入的数据data经过factor对data转化后返回的数据结构，然后run_pipeline可以根据传入的pipe和日期去取对应的数据并进行加工返回最后的dataframe

In [1]:
from quantopian.pipeline import Pipeline

In [2]:
def make_pipeline():
    return Pipeline()

In [3]:
pipe=make_pipeline()

In [4]:
from quantopian.research import run_pipeline

In [5]:
result=run_pipeline(pipe, '2017-01-01','2017-01-01')

In [6]:
result.head()

Empty DataFrame
Columns: []
Index: [(2017-01-03 00:00:00+00:00, Equity(2 [ARNC])), (2017-01-03 00:00:00+00:00, Equity(21 [AAME])), (2017-01-03 00:00:00+00:00, Equity(24 [AAPL])), (2017-01-03 00:00:00+00:00, Equity(25 [ARNC_PR])), (2017-01-03 00:00:00+00:00, Equity(31 [ABAX]))]

In [7]:
result.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8344 entries, (2017-01-03 00:00:00+00:00, Equity(2 [ARNC])) to (2017-01-03 00:00:00+00:00, Equity(50569 [OUSM]))
Empty DataFrame

我们可以看到这是一个多层索引的空dataframe

# Data 

UseEquityPricing包含有股本的价格变化信息

In [8]:
from quantopian.pipeline.data.builtin import USEquityPricing

# Factor 

factor是函数，传入股本的信息asset和相关的时间戳，返回这个factor函数处理后的数值。factor有多个函数，比如有求移动平均的simplemovingaverage，有求权值的平均值EWMA等

In [9]:
from quantopian.pipeline.factors import SimpleMovingAverage,EWMA

In [10]:
def make_pipeline():
    avg_30=SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    return Pipeline(columns={'avg_30':avg_30})

In [11]:
pipe=make_pipeline()
result=run_pipeline(pipe,'2017-01-01','2017-01-01')

result.head()

avg_30
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])       20.110500
                          Equity(21 [AAME])       3.899241
                          Equity(24 [AAPL])     113.368433
                          Equity(25 [ARNC_PR])   86.796111
                          Equity(31 [ABAX])      52.498394

In [12]:
len(result)

8344

In [13]:
def make_pipeline():
    avg_30=SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest=USEquityPricing.close.latest
    return Pipeline(columns={'avg_30':avg_30, 'latest':latest})

In [14]:
pip=make_pipeline()
result=run_pipeline(pip,'2017-01-01','2017-01-01')
result.head()

avg_30  latest
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])       20.110500   18.55
                          Equity(21 [AAME])       3.899241    4.10
                          Equity(24 [AAPL])     113.368433  115.84
                          Equity(25 [ARNC_PR])   86.796111     NaN
                          Equity(31 [ABAX])      52.498394   52.74

In [15]:
def make_pipeline():
    avg_10=SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    avg_30=SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest=USEquityPricing.close.latest
    percent_difference=(avg_10-avg_30)/avg_30
    return Pipeline(columns={'avg_10':avg_10,'avg_30':avg_30,'latest':latest,'percent_difference':percent_difference})

In [16]:
pipe=make_pipeline()
result=run_pipeline(pipe,'2017-01-01','2017-01-01')

result.head()

avg_10      avg_30  latest  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])       19.6530   20.110500   18.55   
                          Equity(21 [AAME])       3.8778    3.899241    4.10   
                          Equity(24 [AAPL])     116.5973  113.368433  115.84   
                          Equity(25 [ARNC_PR])   86.7550   86.796111     NaN   
                          Equity(31 [ABAX])      52.0960   52.498394   52.74   

                                                percent_difference  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])               -0.022749  
                          Equity(21 [AAME])              -0.005499  
                          Equity(24 [AAPL])               0.028481  
                          Equity(25 [ARNC_PR])           -0.000474  
                          Equity(31 [ABAX])              -0.007665

In [17]:
def make_pipeline():
    avg_10=SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    avg_30=SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest=USEquityPricing.close.latest
    percent_difference=(avg_10-avg_30)/avg_30
    check_percent=percent_difference>0
    return Pipeline(columns={'avg_10':avg_10,'avg_30':avg_30,'latest':latest,'percent_difference':percent_difference, 'check_percent':check_percent})

In [18]:
pipe=make_pipeline()
result=run_pipeline(pipe,'2017-01-01','2017-01-01')

result.head()

avg_10      avg_30  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])       19.6530   20.110500   
                          Equity(21 [AAME])       3.8778    3.899241   
                          Equity(24 [AAPL])     116.5973  113.368433   
                          Equity(25 [ARNC_PR])   86.7550   86.796111   
                          Equity(31 [ABAX])      52.0960   52.498394   

                                               check_percent  latest  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])             False   18.55   
                          Equity(21 [AAME])            False    4.10   
                          Equity(24 [AAPL])             True  115.84   
                          Equity(25 [ARNC_PR])         False     NaN   
                          Equity(31 [ABAX])            False   52.74   

                                                percent_difference  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])               -0.022749  
                          Equity(21 [AAME])              -0.005499  
                          Equity(24 [AAPL])               0.028481  
                          Equity(25 [ARNC_PR])           -0.000474  
                          Equity(31 [ABAX])              -0.007665

In [19]:
def make_pipeline():
    avg_10=SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    avg_30=SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest=USEquityPricing.close.latest
    percent_difference=(avg_10-avg_30)/avg_30
    check_percent=percent_difference>0
    return Pipeline(columns={'avg_10':avg_10,'avg_30':avg_30,'latest':latest,'percent_difference':percent_difference, 'check_percent':check_percent},screen=check_percent)

In [20]:
pipe=make_pipeline()
result=run_pipeline(pipe,'2017-01-01','2017-01-01')

result.head()

avg_10      avg_30  \
2017-01-03 00:00:00+00:00 Equity(24 [AAPL])  116.5973  113.368433   
                          Equity(66 [AB])     23.2250   23.119167   
                          Equity(69 [ACAT])   15.9880   15.839500   
                          Equity(70 [VBF])    18.4236   18.208480   
                          Equity(84 [ACET])   21.5440   20.722753   

                                            check_percent  latest  \
2017-01-03 00:00:00+00:00 Equity(24 [AAPL])          True  115.84   
                          Equity(66 [AB])            True   23.45   
                          Equity(69 [ACAT])          True   15.02   
                          Equity(70 [VBF])           True   18.49   
                          Equity(84 [ACET])          True   21.97   

                                             percent_difference  
2017-01-03 00:00:00+00:00 Equity(24 [AAPL])            0.028481  
                          Equity(66 [AB])              0.004578  
                          Equity(69 [ACAT])            0.009375  
                          Equity(70 [VBF])             0.011814  
                          Equity(84 [ACET])            0.039630

In [21]:
def make_pipeline():
    avg_10=SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    avg_30=SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest=USEquityPricing.close.latest
    percent_difference=(avg_10-avg_30)/avg_30
    check_percent=percent_difference>0
    return Pipeline(columns={'avg_10':avg_10,'avg_30':avg_30,'latest':latest,'percent_difference':percent_difference, 'check_percent':check_percent},screen=~check_percent)

In [22]:
pipe=make_pipeline()
result=run_pipeline(pipe,'2017-01-01','2017-01-01')

result.head()

avg_10     avg_30  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])      19.6530  20.110500   
                          Equity(21 [AAME])      3.8778   3.899241   
                          Equity(25 [ARNC_PR])  86.7550  86.796111   
                          Equity(31 [ABAX])     52.0960  52.498394   
                          Equity(39 [DDC])       9.3760   9.523000   

                                               check_percent  latest  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])             False   18.55   
                          Equity(21 [AAME])            False    4.10   
                          Equity(25 [ARNC_PR])         False     NaN   
                          Equity(31 [ABAX])            False   52.74   
                          Equity(39 [DDC])             False    9.69   

                                                percent_difference  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])               -0.022749  
                          Equity(21 [AAME])              -0.005499  
                          Equity(25 [ARNC_PR])           -0.000474  
                          Equity(31 [ABAX])              -0.007665  
                          Equity(39 [DDC])               -0.015436

In [23]:
def make_pipeline():
    avg_10=SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    avg_30=SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest=USEquityPricing.close.latest
    percent_difference=(avg_10-avg_30)/avg_30
    check_percent=percent_difference>0
    small_price=latest<5
    return Pipeline(columns={'avg_10':avg_10,'avg_30':avg_30,'latest':latest,'percent_difference':percent_difference, 'check_percent':check_percent,'small_price':small_price},screen=~check_percent)

In [24]:
pipe=make_pipeline()
result=run_pipeline(pipe,'2017-01-01','2017-01-01')

result.head()

avg_10     avg_30  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])      19.6530  20.110500   
                          Equity(21 [AAME])      3.8778   3.899241   
                          Equity(25 [ARNC_PR])  86.7550  86.796111   
                          Equity(31 [ABAX])     52.0960  52.498394   
                          Equity(39 [DDC])       9.3760   9.523000   

                                               check_percent  latest  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])             False   18.55   
                          Equity(21 [AAME])            False    4.10   
                          Equity(25 [ARNC_PR])         False     NaN   
                          Equity(31 [ABAX])            False   52.74   
                          Equity(39 [DDC])             False    9.69   

                                                percent_difference small_price  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])               -0.022749       False  
                          Equity(21 [AAME])              -0.005499        True  
                          Equity(25 [ARNC_PR])           -0.000474       False  
                          Equity(31 [ABAX])              -0.007665       False  
                          Equity(39 [DDC])               -0.015436       False

# Mask 

In [27]:
def make_pipeline():
    latest=USEquityPricing.close.latest
    small_price=latest<5
    
    avg_10=SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10, mask=small_price)
    avg_30=SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30, mask=small_price)
    
    percent_difference=(avg_10-avg_30)/avg_30
    check_percent=percent_difference>0
    
    final_filter=small_price&check_percent
    return Pipeline(columns={'avg_10':avg_10,'avg_30':avg_30,'latest':latest,'percent_difference':percent_difference, 'check_percent':check_percent,'small_price':small_price},screen=final_filter)
pipe=make_pipeline()
result=run_pipeline(pipe,'2017-01-01','2017-01-01')
result.head()

avg_10    avg_30  \
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])  3.138889  3.097778   
                          Equity(677 [AXAS])  2.595000  2.265333   
                          Equity(1144 [BTX])  3.763500  3.531167   
                          Equity(1323 [CAW])  2.582000  2.541333   
                          Equity(1546 [CIF])  2.539323  2.500370   

                                             check_percent  latest  \
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])          True    3.40   
                          Equity(677 [AXAS])          True    2.56   
                          Equity(1144 [BTX])          True    3.62   
                          Equity(1323 [CAW])          True    2.60   
                          Equity(1546 [CIF])          True    2.57   

                                              percent_difference small_price  
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])            0.013271        True  
                          Equity(677 [AXAS])            0.145527        True  
                          Equity(1144 [BTX])            0.065795        True  
                          Equity(1323 [CAW])            0.016002        True  
                          Equity(1546 [CIF])            0.015579        True

In [28]:
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.classifiers.morningstar import Sector

In [29]:
morningstar_sector = Sector()

In [30]:
exchange = morningstar.share_class_reference.exchange_id.latest

In [31]:
NYS_filter=exchange.eq('NYS')

ArrayPredicate((Latest((share_class_reference.exchange_id::object,), window_length=1),), window_length=0)

In [33]:
def make_pipeline():
    latest=USEquityPricing.close.latest
    small_price=latest<5
    avg_10=SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10, mask=small_price)
    avg_30=SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30, mask=small_price)
    NYS_filter=exchange.eq('NYS')
    percent_difference=(avg_10-avg_30)/avg_30
    check_percent=percent_difference>0
    
    final_filter=small_price&NYS_filter
    return Pipeline(columns={'avg_10':avg_10,'avg_30':avg_30,'latest':latest,'percent_difference':percent_difference, 'check_percent':check_percent,'small_price':small_price},screen=final_filter)
pipe=make_pipeline()
result=run_pipeline(pipe,'2017-01-01','2017-01-01')
result.head()

avg_10    avg_30 check_percent  \
2017-01-03 00:00:00+00:00 Equity(2586 [EQS])  2.0039  1.960533          True   
                          Equity(3265 [GLF])  1.8324  1.576367          True   
                          Equity(3645 [HOV])  2.8325  2.406667          True   
                          Equity(4414 [LEE])  2.9350  3.062500         False   
                          Equity(4577 [LUB])  4.3100  4.292333          True   

                                              latest  percent_difference  \
2017-01-03 00:00:00+00:00 Equity(2586 [EQS])   2.020            0.022120   
                          Equity(3265 [GLF])   1.725            0.162420   
                          Equity(3645 [HOV])   2.735            0.176939   
                          Equity(4414 [LEE])   2.850           -0.041633   
                          Equity(4577 [LUB])   4.270            0.004116   

                                             small_price  
2017-01-03 00:00:00+00:00 Equity(2586 [EQS])        True  
                          Equity(3265 [GLF])        True  
                          Equity(3645 [HOV])        True  
                          Equity(4414 [LEE])        True  
                          Equity(4577 [LUB])        True